In [2]:
import numpy as np
import pandas as pd

import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv1D, BatchNormalization, Dropout, MaxPooling1D

np.random.seed(42)

import warnings
warnings.filterwarnings('ignore')

## Import data

In [81]:
file_name = './dataset/feature_sel.csv'
df = pd.read_csv(file_name)

##### cleaning

In [82]:
df.year = pd.to_datetime(df.year.str.strip()) 
df = df.dropna().drop_duplicates()
print(f'df shape: {df.shape}'); print(f'df cols: {df.columns}')

df shape: (314, 28)
df cols: Index(['Unnamed: 0', 'name', 'juristicid', 'year', 'accountreceivable',
       'accountpayable', 'totalnoncurrentliability', 'totalliability',
       'totalcurrentliability', 'totalcurrentasset', 'totalasset',
       'totalexpense', 'cashandcashequivalent', 'otherincome', 'totalrevenue',
       'netprofitloss', 'pct_accountreceivable', 'pct_accountpayable',
       'pct_totalnoncurrentliability', 'pct_totalliability',
       'pct_totalcurrentliability', 'pct_totalcurrentasset', 'pct_totalasset',
       'pct_totalexpense', 'pct_cashandcashequivalent', 'pct_otherincome',
       'pct_totalrevenue', 'pct_netprofitloss'],
      dtype='object')


In [83]:
df = df.loc[(df.year == '2014-12-31') | (df.year == '2022-12-31')]
df = df.drop(axis=1, columns=['Unnamed: 0', 'name', 'juristicid', 'year'])

## Feature Engineering

In [84]:
df_ptt_2014 = df.iloc[0]
df_ptt_2022 = df.iloc[1]

In [85]:
ppt_2014 = np.array(df_ptt_2014).reshape((12, 2), order='F')
print(f'X shape: {ppt_2014.shape}')
ppt_2022 = np.array(df_ptt_2022.iloc[:12])
print(f'y shape: {ppt_2022.shape}')

X shape: (12, 2)
y shape: (12,)


## QNet

In [155]:
X = ppt_2014[np.newaxis, : , :]
y = ppt_2022[np.newaxis, :, np.newaxis]

In [156]:
X.shape, y.shape

((1, 12, 2), (1, 12, 1))

In [157]:
model = Sequential()

model.add(keras.layers.Input(shape=(13, 2)))

model.add(Conv1D(filters=4, kernel_size=1, padding='same', strides=1, data_format='channels_last'))
model.add(BatchNormalization())
model.add(MaxPooling1D(pool_size=1))
model.add(Dropout(0.2))

model.add(Conv1D(filters=2, kernel_size=1, padding='same', strides=1, data_format='channels_last'))
model.add(BatchNormalization())
model.add(MaxPooling1D(pool_size=1))
model.add(Dropout(0.2))

model.add(Dense(1))

opt = keras.optimizers.Adam(learning_rate=1e-4)
model.compile(optimizer=opt, loss='mean_squared_error')

model.summary()

Model: "sequential_37"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_16 (Conv1D)              │ (None, 13, 4)          │            12 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_74          │ (None, 13, 4)          │            16 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_16 (MaxPooling1D) │ (None, 13, 4)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 13, 4)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_17 (Conv1D)              │ (None, 13, 2)          │            10 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_75          │ (None, 13, 2)          │             8 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_17 (MaxPooling1D) │ (None, 13, 2)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 13, 2)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_37 (Dense)                │ (None, 13, 1)          │             3 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 49 (196.00 B)

 Trainable params: 37 (148.00 B)

 Non-trainable params: 12 (48.00 B)

In [158]:
model.evaluate(X, y)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 115ms/step - loss: 99793633839837712744448.0000


9.979363383983771e+22